<a href="https://colab.research.google.com/github/mr-alamdari/NLP-Parts-of-Speech-Tagging-Begginer/blob/main/NLP_Parts_of_Speech_Tagging_Begginer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import os
import re
import collections
import numpy as np
import pandas as pd

In [3]:
!wget https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/WSJ_24.pos

--2022-04-15 12:00:45--  https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/WSJ_24.pos
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 286063 (279K) [text/plain]
Saving to: ‘WSJ_24.pos.1’

WSJ_24.pos.1        100%[===================>] 279.36K  --.-KB/s    in 0.02s   

2022-04-15 12:00:45 (12.4 MB/s) - ‘WSJ_24.pos.1’ saved [286063/286063]



In [4]:
!wget https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/WSJ_02-21.pos

--2022-04-15 12:00:46--  https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/WSJ_02-21.pos
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8279089 (7.9M) [text/plain]
Saving to: ‘WSJ_02-21.pos.1’

WSJ_02-21.pos.1     100%[===================>]   7.90M  --.-KB/s    in 0.07s   

2022-04-15 12:00:46 (115 MB/s) - ‘WSJ_02-21.pos.1’ saved [8279089/8279089]



In [5]:
!wget https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/hmm_vocab.txt

--2022-04-15 12:03:04--  https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/hmm_vocab.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 196571 (192K) [text/plain]
Saving to: ‘hmm_vocab.txt’

hmm_vocab.txt       100%[===================>] 191.96K  --.-KB/s    in 0.02s   

2022-04-15 12:03:04 (10.4 MB/s) - ‘hmm_vocab.txt’ saved [196571/196571]



In [6]:
with open('WSJ_02-21.pos', 'r') as f:
  train_corpus = f.readlines()

In [8]:
train_corpus[10: 20]

['at\tIN\n',
 'Chicago\tNNP\n',
 "'s\tPOS\n",
 'Goodman\tNNP\n',
 'Theatre\tNNP\n',
 '(\t(\n',
 '``\t``\n',
 'Revitalized\tVBN\n',
 'Classics\tNNS\n',
 'Take\tVBP\n']

In [9]:
with open('WSJ_24.pos', 'r') as f:
  y = f.readlines()

In [10]:
y[10: 20]

['points\tNNS\n',
 'this\tDT\n',
 'week\tNN\n',
 ',\t,\n',
 'with\tIN\n',
 'readings\tNNS\n',
 'on\tIN\n',
 'trade\tNN\n',
 ',\t,\n',
 'output\tNN\n']

In [20]:
with open('hmm_vocab.txt', 'r') as f:
  vocabulary_list = f.read().split('\n')

vocab = {}
for ind, word in enumerate(sorted(vocabulary_list)):
  vocab[word] = ind

In [21]:
sorted(vocabulary_list)[10: 20]

["'70s", "'80s", "'86", "'90s", "'N", "'S", "'d", "'em", "'ll", "'m"]

In [27]:
for id, (k, v) in enumerate(vocab.items()):
  print(k, v)
  if id == 10:
    break

 0
! 1
# 2
$ 3
% 4
& 5
' 6
'' 7
'40s 8
'60s 9
'70s 10


In [76]:
# https://github.com/amanjeetsahu/Natural-Language-Processing-Specialization/blob/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/utils_pos.py

import string


# Punctuation characters
punct = set(string.punctuation)

# Morphology rules used to assign unknown word tokens
noun_suffix = ["action", "age", "ance", "cy", "dom", "ee", "ence", "er", "hood", "ion", "ism", "ist", "ity", "ling", "ment", "ness", "or", "ry", "scape", "ship", "ty"]
verb_suffix = ["ate", "ify", "ise", "ize"]
adj_suffix = ["able", "ese", "ful", "i", "ian", "ible", "ic", "ish", "ive", "less", "ly", "ous"]
adv_suffix = ["ward", "wards", "wise"]



def word_tag_spliter(word_tag, vocab):

  if word_tag.split():
    word, tag = word_tag.split()
    if not vocab.get(word):
      word = assign_unk(word)
  else:
      word, tag = '--n--', '--s--'
  
  return word, tag

def assign_unk(tok):
    """
    Assign unknown word tokens
    """
    # Digits
    if any(char.isdigit() for char in tok):
        return "--unk_digit--"

    # Punctuation
    elif any(char in punct for char in tok):
        return "--unk_punct--"

    # Upper-case
    elif any(char.isupper() for char in tok):
        return "--unk_upper--"

    # Nouns
    elif any(tok.endswith(suffix) for suffix in noun_suffix):
        return "--unk_noun--"

    # Verbs
    elif any(tok.endswith(suffix) for suffix in verb_suffix):
        return "--unk_verb--"

    # Adjectives
    elif any(tok.endswith(suffix) for suffix in adj_suffix):
        return "--unk_adj--"

    # Adverbs
    elif any(tok.endswith(suffix) for suffix in adv_suffix):
        return "--unk_adv--"

    return "--unk--"

def preprocess(vocab, data_fp):
    """
    Preprocess data
    """
    orig = []
    prep = []

    with open(data_fp, "r") as data_file:
        for cnt, word in enumerate(data_file):
            # End of sentence
            if not word.split():
                orig.append(word.strip())
                word = "--n--"
                prep.append(word)
                continue

            # Handle unknown words
            elif word.strip() not in vocab:
                orig.append(word.strip())
                word = assign_unk(word)
                prep.append(word)
                continue

            else:
                orig.append(word.strip())
                prep.append(word.strip())

    assert(len(orig) == len(open(data_fp, "r").readlines()))
    assert(len(prep) == len(open(data_fp, "r").readlines()))

    return orig, prep


In [29]:
!wget https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/test.words

--2022-04-15 12:43:47--  https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/test.words
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 180266 (176K) [text/plain]
Saving to: ‘test.words’

test.words          100%[===================>] 176.04K  --.-KB/s    in 0.01s   

2022-04-15 12:43:47 (11.6 MB/s) - ‘test.words’ saved [180266/180266]



In [77]:
_, prep = preprocess(vocab, 'test.words')

In [78]:
prep[10:20]

['points',
 'this',
 'week',
 ',',
 'with',
 'readings',
 'on',
 'trade',
 ',',
 'output']

In [79]:
from collections import defaultdict

def dictionary_creator(corpus, vocab):
  emission_counts = defaultdict(int)
  transition_counts = defaultdict(int)
  tag_counts = defaultdict(int)
  prev_tag = '--s--'
  for word_tag in corpus:
    word, tag = word_tag_spliter(word_tag, vocab)

    transition_counts[(prev_tag, tag)] += 1
    emission_counts[(tag, word)] += 1
    tag_counts[tag] += 1
    prev_tag = tag
  
  return emission_counts, transition_counts, tag_counts 

In [80]:
emission_counts, transition_counts, tag_counts = dictionary_creator(train_corpus, vocab)

In [90]:
for i, item in enumerate(emission_counts.items()):
  print(item[0], item[1])
  if i == 20:
    break

('IN', 'In') 1735
('DT', 'an') 3142
('NNP', 'Oct.') 317
('CD', '19') 100
('NN', 'review') 36
('IN', 'of') 22925
('``', '``') 6967
('DT', 'The') 6795
('NN', 'Misanthrope') 3
("''", "''") 6787
('IN', 'at') 4361
('NNP', 'Chicago') 197
('POS', "'s") 8079
('NNP', 'Goodman') 7
('NNP', 'Theatre') 5
('(', '(') 1153
('VBN', '--unk_upper--') 93
('NNS', '--unk_upper--') 350
('VBP', 'Take') 1
('DT', 'the') 41098
('NN', 'Stage') 1


In [91]:
for i, item in enumerate(transition_counts.items()):
  print(item[0], item[1])
  if i == 20:
    break

('--s--', 'IN') 5050
('IN', 'DT') 32364
('DT', 'NNP') 9044
('NNP', 'CD') 1752
('CD', 'NN') 7377
('NN', 'IN') 32885
('IN', '``') 546
('``', 'DT') 1014
('DT', 'NN') 38873
('NN', "''") 686
("''", 'IN') 591


In [92]:
for i, item in enumerate(tag_counts.items()):
  print(item[0], item[1])
  if i == 20:
    break

IN 98554
DT 81842
NNP 91466
CD 36568
NN 132935
`` 7092
'' 6919
POS 8701
( 1366
VBN 20024
NNS 59856
VBP 12491
, 48727
CC 23947
) 1376
VBD 29889
RB 30970
TO 22357
. 39478
--s-- 39832
VBZ 21672


In [121]:
def pos_predictor(prep, y, emission_counts, vocab, states): 
  corrects = 0
  total = len(y)

  for word, word_pos in zip(prep, y):
    
    if len(word_pos) == 1:
      continue

    true_word, true_pos = word_pos.split()
    final_count, final_pos = 0, ''
 
    if vocab.get(word):
      for pos in states:
        
        key = (pos, word)
        count = emission_counts.get(key)
        
        if count != None and count > final_count:
          final_count = count
          final_pos = pos
      
      if final_pos == true_pos:
        corrects += 1

  accuracy = corrects / total
  
  return accuracy

In [136]:
vocab

{'': 0,
 '!': 1,
 '#': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 "''": 7,
 "'40s": 8,
 "'60s": 9,
 "'70s": 10,
 "'80s": 11,
 "'86": 12,
 "'90s": 13,
 "'N": 14,
 "'S": 15,
 "'d": 16,
 "'em": 17,
 "'ll": 18,
 "'m": 19,
 "'n'": 20,
 "'re": 21,
 "'s": 22,
 "'til": 23,
 "'ve": 24,
 '(': 25,
 ')': 26,
 ',': 27,
 '-': 28,
 '--': 29,
 '--n--': 30,
 '--unk--': 31,
 '--unk_adj--': 32,
 '--unk_adv--': 33,
 '--unk_digit--': 34,
 '--unk_noun--': 35,
 '--unk_punct--': 36,
 '--unk_upper--': 37,
 '--unk_verb--': 38,
 '.': 39,
 '...': 40,
 '0.01': 41,
 '0.0108': 42,
 '0.02': 43,
 '0.03': 44,
 '0.05': 45,
 '0.1': 46,
 '0.10': 47,
 '0.12': 48,
 '0.13': 49,
 '0.15': 50,
 '0.17': 51,
 '0.19': 52,
 '0.2': 53,
 '0.24': 54,
 '0.25': 55,
 '0.3': 56,
 '0.32': 57,
 '0.375': 58,
 '0.4': 59,
 '0.43': 60,
 '0.45': 61,
 '0.5': 62,
 '0.53': 63,
 '0.59': 64,
 '0.6': 65,
 '0.60': 66,
 '0.7': 67,
 '0.71': 68,
 '0.75': 69,
 '0.8': 70,
 '0.88': 71,
 '0.9': 72,
 '0.94': 73,
 '0.95': 74,
 '1': 75,
 '1,000': 76,
 '1,012': 77,


In [122]:
pos_predictor(prep, y, emission_counts, vocab, states)

Streaming output truncated to the last 5000 lines.
NN 101
DT 1168
IN 4815
MD 193
RB 3222
VB 3724
RB 17
VBD 5
IN 4483
PRP$ 1678
NNS 43
. 39020
DT 6795
NN 195
NNP 1
POS 8079
NN 194
VBD 3903
VBD 2
VBN 4
FW 2
IN 14957
DT 19264
$ 1
CD 4
JJ 1942
JJ 20
VBD 81
VBN 111
FW 2
IN 14957
CD 1220
IN 22925
CD 591
NN 101
NNS 139
DT 41098
NNP 82
NN 3
NNP 130
MD 3055
VB 27
VBD 34
CC 16113
MD 3055
VB 42
DT 1897
NN 88
. 39020
DT 6795
NN 195
VBD 81
VBN 111
DT 41098
JJ 20
FW 2
IN 14957
, 1
DT 3142
NN 39
VBG 58
DT 19264
NNP 14
NN 298
DT 1168
IN 4815
VBZ 34
DT 1168
IN 4815
CC 25
DT 336
NNS 43
IN 22925
DT 19264
JJ 51
VB 3724
VBD 3
VBN 10
IN 469
PRP 384
MD 808
JJ 1
VB 1108
VBP 1932
, 1
DT 3142
NN 101
. 39020
DT 6795
NN 195
VBD 31
DT 41098
NNS 3
TO 22198
VB 19
DT 19264
JJ 148
JJ 53
JJ 1
VBD 31
IN 4483
JJ 1
NNP 20
NN 3
NNP 80
NNP 126
CD 5
FW 12
IN 13
JJ 244
NN 365
NNP 4635
JJ 1007
NNP 193
WRB 987
DT 41098
JJ 315
NN 274
VBD 6
NNP 20
NNP 1
POS 8079
NN 101
NNS 64
. 39020
IN 136
DT 1168
IN 4815
JJ 53
, 48723
WDT 2218


0.8888563993099213

['The\tDT\n',
 'economy\tNN\n',
 "'s\tPOS\n",
 'temperature\tNN\n',
 'will\tMD\n',
 'be\tVB\n',
 'taken\tVBN\n',
 'from\tIN\n',
 'several\tJJ\n',
 'vantage\tNN\n',
 'points\tNNS\n',
 'this\tDT\n',
 'week\tNN\n',
 ',\t,\n',
 'with\tIN\n',
 'readings\tNNS\n',
 'on\tIN\n',
 'trade\tNN\n',
 ',\t,\n',
 'output\tNN\n',
 ',\t,\n',
 'housing\tNN\n',
 'and\tCC\n',
 'inflation\tNN\n',
 '.\t.\n',
 '\n',
 'The\tDT\n',
 'most\tRBS\n',
 'troublesome\tJJ\n',
 'report\tNN\n',
 'may\tMD\n',
 'be\tVB\n',
 'the\tDT\n',
 'August\tNNP\n',
 'merchandise\tNN\n',
 'trade\tNN\n',
 'deficit\tNN\n',
 'due\tJJ\n',
 'out\tIN\n',
 'tomorrow\tNN\n',
 '.\t.\n',
 '\n',
 'The\tDT\n',
 'trade\tNN\n',
 'gap\tNN\n',
 'is\tVBZ\n',
 'expected\tVBN\n',
 'to\tTO\n',
 'widen\tVB\n',
 'to\tTO\n',
 'about\tIN\n',
 '$\t$\n',
 '9\tCD\n',
 'billion\tCD\n',
 'from\tIN\n',
 'July\tNNP\n',
 "'s\tPOS\n",
 '$\t$\n',
 '7.6\tCD\n',
 'billion\tCD\n',
 ',\t,\n',
 'according\tVBG\n',
 'to\tTO\n',
 'a\tDT\n',
 'survey\tNN\n',
 'by\tIN\n',
 'M

In [151]:
def transition_matrix(transition_counts, tag_counts, alpha):
  
  all_tags = sorted(tag_counts.keys())
  tags_count = len(all_tags)
  a = np.zeros((tags_count, tags_count))
  transition_krys = set(transition_counts.keys())

  for i in range(tags_count):
    for j in range(tags_count):
      key = (all_tags[i], all_tags[j])
      
      tr_count = transition_counts.get(key)

      count = tr_count if tr_count else 0

      prev_tag_count = tag_counts.get(all_tags[i])

      a[i, j] = (count+alpha) / (prev_tag_count + tags_count*alpha)
  
  return a

In [196]:
a = transition_matrix(transition_counts, tag_counts, 0.001)

In [197]:
a

array([[7.03997297e-06, 7.03997297e-06, 7.03997297e-06, ...,
        7.03997297e-06, 7.03997297e-06, 7.03997297e-06],
       [1.35647553e-07, 1.35647553e-07, 1.35647553e-07, ...,
        1.35647553e-07, 1.35647553e-07, 1.35647553e-07],
       [1.44528595e-07, 1.44673124e-04, 6.93751711e-03, ...,
        2.89201719e-04, 3.17977363e-03, 2.74618784e-03],
       ...,
       [5.95075158e-06, 2.38089571e-02, 5.95075158e-06, ...,
        5.95075158e-06, 5.95075158e-06, 5.95670233e-03],
       [4.66625541e-07, 4.67092167e-04, 4.66625541e-07, ...,
        4.66625541e-07, 4.67092167e-04, 1.86696879e-03],
       [1.41003034e-07, 1.41144037e-04, 1.41003034e-07, ...,
        1.41003034e-07, 1.12803837e-02, 4.23150104e-04]])

In [189]:
def emission_matrix(emission_counts, tag_counts, vocab, alpha):
  tags_count = len(tag_counts)
  all_tags = sorted(tag_counts.keys())
  vocab_list = list(vocab.keys())
  words_count = len(vocab_list)
  emission_keys = set(list(emission_counts.keys()))

  b = np.zeros((tags_count, words_count))

  for i in range(tags_count):
    for j in range(words_count):

      word = vocab_list[j]
      key = (all_tags[i], vocab.get(word))
      ems_key = emission_counts.get(key)

      count = ems_key if ems_key else 0
      count_tag = tag_counts.get(all_tags[i])
      b[i, j] = (count+alpha) / (count_tag + alpha*words_count)
  
  return b

In [190]:
b = emission_matrix(emission_counts, tag_counts, vocab, 0.001)

In [191]:
b

array([[6.03219988e-06, 6.03219988e-06, 6.03219988e-06, ...,
        6.03219988e-06, 6.03219988e-06, 6.03219988e-06],
       [1.35212298e-07, 1.35212298e-07, 1.35212298e-07, ...,
        1.35212298e-07, 1.35212298e-07, 1.35212298e-07],
       [1.44034584e-07, 1.44034584e-07, 1.44034584e-07, ...,
        1.44034584e-07, 1.44034584e-07, 1.44034584e-07],
       ...,
       [5.21438963e-06, 5.21438963e-06, 5.21438963e-06, ...,
        5.21438963e-06, 5.21438963e-06, 5.21438963e-06],
       [4.61514960e-07, 4.61514960e-07, 4.61514960e-07, ...,
        4.61514960e-07, 4.61514960e-07, 4.61514960e-07],
       [1.40532791e-07, 1.40532791e-07, 1.40532791e-07, ...,
        1.40532791e-07, 1.40532791e-07, 1.40532791e-07]])

In [194]:
def initialization(states, tag_counts, a, b, corpus, vocab):
  tags_count = len(tag_counts)
  best_probs = np.zeros((tags_count, len(corpus)))
  best_paths = np.zeros((tags_count, len(corpus)), dtype=int)
  s_ind = states.index('--s--')

  for i in range(tags_count):
    best_probs[i, 0] = float('-inf') if a[s_ind, i]==0 else np.log(a[s_ind,i]) + np.log(b[i,vocab[corpus[0]]])

  return best_probs, best_paths

In [200]:
best_probs, best_paths = initialization(states, tag_counts, a, b, prep, vocab)

In [201]:
best_probs

array([[-22.60982633,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [-23.07660654,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [-23.57298822,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       ...,
       [-22.75551606,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [-19.6637215 ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [-18.36288463,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ]])

In [202]:
best_paths

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])